<a href="https://colab.research.google.com/github/pinkesh-ai/DL201/blob/main/10_PPAi21V2_nlp_own_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Deep Dive - Own Code

Own refactored code and notes for *Chapter 10: NLP Deep Dive: RNNs* ([`10_nlp.ipynb`](https://colab.research.google.com/github/vtecftwy/fastbook/blob/master/10_nlp.ipynb)).

## Instructions

It is recommended that you work in two steps:
1. Copy the code from the fastbook notebook and make sure it works
2. Refactor (i.e. rewrite the code in your own style) by 
    - regrouping things together that make sense ro you
    - adding text cells to explain what to code does in your own words and possible references to the doc you may have consulsted
    - deleting code you think was only there to explain things but are not required once you run models end to end

When you have done that, you get a customized reference notebook for you which you can consult later on when you forgot the details, withouht having to read the full notebook from fastbook.

10: NLP - Patel 

In [3]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 8.4MB/s 
     |████████████████████████████████| 194kB 40.1MB/s 
     |████████████████████████████████| 1.2MB 51.4MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
     |████████████████████████████████| 61kB 10.3MB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter you

In [4]:
#hide
from fastbook import *
from IPython.display import display,HTML

In [5]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [ ]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [ ]:
txt = files[0].open().read(); txt[:75]

In [8]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#310) ['Given','the','history','of','the','director','of','this','movie',',','it','is','hard','to','believe','that','this','was','such','a','painfully','bad','movie','to','sit','through','.','I','was','at'...]


In [9]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [10]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#328) ['xxbos','xxmaj','given','the','history','of','the','director','of','this','movie',',','it','is','hard','to','believe','that','this','was','such','a','painfully','bad','movie','to','sit','through','.','i','was'...]


In [11]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#328) ['xxbos','xxmaj','given','the','history','of','the','director','of','this','movie',',','it','is','hard','to','believe','that','this','was','such','a','painfully','bad','movie','to','sit','through','.','i','was'...]


Default Rules 

In [12]:
txts = L(o.open().read() for o in files[:2000])

In [13]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [14]:
subword(1000)

'▁G ive n ▁the ▁his t or y ▁of ▁the ▁director ▁of ▁this ▁movie , ▁it ▁is ▁hard ▁to ▁believe ▁that ▁this ▁was ▁such ▁a ▁pa in ful ly ▁bad ▁movie ▁to ▁s it ▁through . ▁I ▁was ▁at ▁the'

In [15]:
subword(200)

'▁ G i ve n ▁the ▁h is t or y ▁of ▁the ▁d i re c t or ▁of ▁this ▁movie , ▁it ▁is ▁h ar d ▁to ▁be li e ve ▁that ▁this ▁was ▁s u ch ▁a'

In [ ]:
subword(10000)

In [ ]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

In [ ]:
toks200 = txts[:200].map(tkn)
toks200[0]

In [ ]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

In [ ]:
nums = num(toks)[:20]; nums

In [ ]:
' '.join(num.vocab[o] for o in nums)

Batches =No Resize 

In [22]:
nums200 = toks200.map(num)

In [ ]:
dl = LMDataLoader(nums200)

In [ ]:
x,y = first(dl)
x.shape,y.shape

In [ ]:
' '.join(num.vocab[o] for o in x[0][:20])

In [ ]:
' '.join(num.vocab[o] for o in y[0][:20])

In [27]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])
dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [28]:
dls_lm.show_batch(max_n=2)

,text,text_
0,xxbos xxmaj is torture ever right ? xxmaj no xxmaj the answer is simple and absolute with no qualifications possible . xxmaj the reason as this film showed is the effect torture has on a society . xxmaj the values that have been hard fought for in xxmaj western society through centuries of revolution and struggle are for xxup all men and women to be allowed to live in a free and open society . xxmaj one where individuals are,xxmaj is torture ever right ? xxmaj no xxmaj the answer is simple and absolute with no qualifications possible . xxmaj the reason as this film showed is the effect torture has on a society . xxmaj the values that have been hard fought for in xxmaj western society through centuries of revolution and struggle are for xxup all men and women to be allowed to live in a free and open society . xxmaj one where individuals are treated
1,"death scenes such as a man falling in a wood chipper and a sword through the neck which are … . well , sucky . i must warn you to stay away from this movie , for it is not worthy of being viewed . xxmaj crap crap crap ! xxbos a very great movie . \n\n a big love story . xxmaj lots of sword fighting . xxmaj huge battle scenes . xxmaj heros and villains . xxmaj real","scenes such as a man falling in a wood chipper and a sword through the neck which are … . well , sucky . i must warn you to stay away from this movie , for it is not worthy of being viewed . xxmaj crap crap crap ! xxbos a very great movie . \n\n a big love story . xxmaj lots of sword fighting . xxmaj huge battle scenes . xxmaj heros and villains . xxmaj real history"


In [29]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time


In [ ]:
learn.save('1epoch')

In [ ]:
learn = learn.load('1epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, 2e-3)

In [ ]:
learn.save_encoder('finetuned')

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [ ]:
dls_clas.show_batch(max_n=3)

In [ ]:
nums_samp = toks200[:10].map(num)

In [ ]:
nums_samp.map(len)

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [ ]:
learn = learn.load_encoder('finetuned')

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

In [ ]:
pretrained model+ fine-tuned it for task+ body (the encoder)+ New Head( Encoder) 